Imports

In [7]:
import ccxt
import pandas as pd
import plotly.graph_objects as go

Fetch the data using ccxt

In [8]:
exchange = ccxt.binance()
symbol = 'BTC/USDT'
candles = exchange.fetch_ohlcv(symbol, '1h')
df = pd.DataFrame(candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

print(df.head())

            timestamp      open      high       low     close      volume
0 2023-09-25 15:00:00  26133.89  26300.00  26133.89  26243.72  1709.30587
1 2023-09-25 16:00:00  26243.73  26425.63  26243.72  26324.78  2219.03348
2 2023-09-25 17:00:00  26324.77  26375.87  26222.32  26358.41  1671.84321
3 2023-09-25 18:00:00  26358.41  26389.09  26315.58  26331.58   883.54843
4 2023-09-25 19:00:00  26331.58  26352.94  26269.59  26331.83   900.19334


Functions to determine support and resistance

In [9]:
def support(df1, l, n1, n2):
    # For the range before the current candle:
    for i in range(max(0, l-n1+1), l+1):
        # Ensure we're not checking for an index < 0
        if i-1 < 0:
            continue
        if df1.low[i] > df1.low[i-1]:
            return 0

    # For the range after the current candle:
    for i in range(l+1, min(l+n2+1, len(df1))):
        # Ensure we're not checking for an index beyond DataFrame's end
        if i-1 >= len(df1):
            continue
        if df1.low[i] < df1.low[i-1]:
            return 0

    return 1


def resistance(df1, l, n1, n2):
    # For the range before the current candle:
    for i in range(max(0, l-n1+1), l+1):
        # Ensure we're not checking for an index < 0
        if i-1 < 0:
            continue
        if df1.high[i] < df1.high[i-1]:
            return 0

    # For the range after the current candle:
    for i in range(l+1, min(l+n2+1, len(df1))):
        # Ensure we're not checking for an index beyond DataFrame's end
        if i-1 >= len(df1):
            continue
        if df1.high[i] > df1.high[i-1]:
            return 0

    return 1

Find support and resistance

In [10]:
df['support'] = df.apply(lambda row: support(df, row.name, 4, 4), axis=1)
df['resistance'] = df.apply(lambda row: resistance(df, row.name, 4, 4), axis=1)

Plot candlestick chart with support and resistance lines

In [14]:
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

fig = go.Figure(data=[go.Candlestick(x=df['timestamp'],
                                     open=df['open'],
                                     high=df['high'],
                                     low=df['low'],
                                     close=df['close'])])

for index, row in df.iterrows():
    if row['support'] == 1:
        fig.add_shape(type='line',
                      x0=row['timestamp'],
                      # last timestamp, you can also set it to some fixed value for specific range
                      x1=df['timestamp'].iloc[-1],
                      y0=row['low'],
                      y1=row['low'],
                      line=dict(color='green'))
    if row['resistance'] == 1:
        fig.add_shape(type='line',
                      x0=row['timestamp'],
                      x1=df['timestamp'].iloc[-1],  # last timestamp
                      y0=row['high'],
                      y1=row['high'],
                      line=dict(color='red'))
        
    fig.update_layout(height=1200, template="plotly_dark")

fig.show()
